In [1]:
!unzip "/content/drive/MyDrive/rice.zip"

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [3]:
# Define constants
num_classes = 2
train_data_dir = r'C:\Users\user\Desktop\python training code\product defective 50 percent\train'
validation_data_dir = r'C:\Users\user\Desktop\python training code\product defective 50 percent\validation'
img_height, img_width = 640, 640
batch_size = 5
num_train_samples = 350
num_validation_samples = 100
epochs = 6

In [4]:
# Load Xception model
base_model = Xception(weights='imagenet', include_top=False)

# Add custom output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Add a final softmax layer for classification

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Preprocess the input data
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input)

In [5]:

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1300 images belonging to 2 classes.
Found 1300 images belonging to 2 classes.


In [ ]:
# Train the model
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)


Epoch 1/6
260/260 [==============================] - 55529s 214s/step - loss: 0.6722 - accuracy: 0.6000 - val_loss: 0.6729 - val_accuracy: 0.6008
Epoch 2/6
260/260 [==============================] - 3439s 13s/step - loss: 0.6766 - accuracy: 0.5977 - val_loss: 0.6733 - val_accuracy: 0.6008
Epoch 3/6
260/260 [==============================] - 3036s 12s/step - loss: 0.6752 - accuracy: 0.6008 - val_loss: 0.6732 - val_accuracy: 0.6008
Epoch 4/6
260/260 [==============================] - ETA: 0s - loss: 0.6754 - accuracy: 0.6008 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
# Prediction
Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
# Get class labels
class_labels = list(validation_generator.class_indices.keys())

In [ ]:
# Print classification report
print("Classification Report:")
print(classification_report(validation_generator.classes, y_pred, target_names=class_labels))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(validation_generator.classes, y_pred))

In [ ]:
# Save the trained model
model.save('xceptionv_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('xceptionv_model.h5')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Load the saved model
loaded_model = tf.keras.models.load_model('xceptionv_model.h5')

# Load and preprocess the input image
img_path = '/content/bacterial_leaf_blight (1).JPG'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = tf.keras.applications.inception_v3.preprocess_input(img_array)

# Make prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the class name corresponding to the predicted class index
predicted_class_name = class_labels[predicted_class]

# Display the input image
plt.imshow(img)
plt.axis('off')
plt.title('Input Image')
plt.show()

# Print predicted class index and its corresponding class name
print("Predicted Class Index:", predicted_class)
print("Predicted Class Name:", predicted_class_name)
